# PyKX Query Components
Query all components, RDB, HDB, and Gateway.


In [1]:
# clusters
RDB_CLUSTER_NAME="RDB_basictickdb_20230606"
HDB_CLUSTER_NAME="HDB_basictickdb_20230606"
GW_CLUSTER_NAME ="GATEWAY_20230606"


In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import os
import boto3
import json
import datetime

import pykx as kx

from managed_kx import *
from env_2 import *


In [3]:
# triggers credential get
session=None

try:
    # aws: use ada for credentials
    subprocess.call(["which", "ada"])
    os.system(f"ada credentials update --account={ACCOUNT_ID} --provider=isengard --role=Admin --once")
except: 
    None

if AWS_ACCESS_KEY_ID is None:
    print("Using Defaults ...")
    # create AWS session: using access variables
    session = boto3.Session()
else:
    print("Using variables ...")
    session = boto3.Session(
        aws_access_key_id=AWS_ACCESS_KEY_ID,
        aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
        aws_session_token=AWS_SESSION_TOKEN
    )

# create finspace client
client = session.client(service_name='finspace', endpoint_url=ENDPOINT_URL)

Using variables ...


# Current State of HDB

In [4]:
# Query the HDB
hdb = get_pykx_connection(client, 
                          environmentId=ENV_ID, clusterName=HDB_CLUSTER_NAME, 
                          userName=KDB_USERNAME, boto_session=session)

# Dates and Counts
hdb_pdf = hdb("select counts:count i by date from example").pd()

# Number of Rows
hdb_rows = hdb("count example").py()

# Current State of RDB

In [5]:
# Query the RDB
rdb = get_pykx_connection(client, 
                          environmentId=ENV_ID, clusterName=RDB_CLUSTER_NAME, 
                          userName=KDB_USERNAME, boto_session=session)

# Dates and Counts
rdb_head_pdf = rdb("select [5] from example").pd()
rdb_tail_pdf = rdb("select [-5] from example").pd()

# Number of Rows
rdb_rows = rdb("count example").py()

# Current State of GW

In [6]:
# Query the HDB
gw = get_pykx_connection(client, 
                          environmentId=ENV_ID, clusterName=GW_CLUSTER_NAME, 
                          userName=KDB_USERNAME, boto_session=session)

In [7]:
proc_pdf = gw("select process, handle, connected, address from .conn.procs").pd()

# are any processes not connected?
if len(proc_pdf[proc_pdf.connected == False].index) > 0:
    gw("reinit[hdb_name; rdb_name]")
    proc_pdf = gw("select process, handle, connected, address from .conn.procs").pd()

# truncate with elipsis
proc_pdf['address'] = proc_pdf['address'].str.slice(0,110)+"..."

display(proc_pdf)

,process,handle,connected,address
0,rdb,42,True,:ip-192-168-2-98.ec2.internal:5000:GATEWAY_20230606:Host=ip-192-168-2-98.ec2.internal&Port=5000&User...
1,hdb,43,True,:ip-192-168-6-13.ec2.internal:5000:GATEWAY_20230606:Host=ip-192-168-6-13.ec2.internal&Port=5000&User...
2,hdb,44,True,:ip-192-168-4-47.ec2.internal:5000:GATEWAY_20230606:Host=ip-192-168-4-47.ec2.internal&Port=5000&User...


In [8]:
gw("res: `time xasc queryData[`example;`]").pd()

gw_pdf = gw("res").pd()
gw_head_pdf = gw("select [5] from res").pd()
gw_tail_pdf = gw("select [-5] from res").pd()

# Number of Rows
gw_rows = len(gw_pdf.index)

# HDB Contents

In [9]:
display(hdb_pdf)

# Number of Rows
print(f"Rows: {hdb_rows:,}")

,counts
date,
2023-04-14,1000000
2023-04-15,1000000
2023-04-16,1000000
2023-04-17,1000000
2023-04-18,1000000
2023-04-19,1000000
2023-04-20,1000000
2023-04-21,1000000
2023-04-22,1000000


Rows: 25,998,183


# RDB Contents

In [10]:
display(rdb_head_pdf)
display(rdb_tail_pdf)

# Number of Rows
print(f"Rows: {rdb_rows:,}")

,sym,time,number
0,lgf,2023-06-12 14:51:53.924018353,1
1,mfe,2023-06-12 14:51:53.924018353,23
2,ofp,2023-06-12 14:51:53.924018353,53
3,bmp,2023-06-12 14:51:53.924018353,57
4,png,2023-06-12 14:51:53.924018353,81


,sym,time,number
0,mpi,2023-06-12 20:14:38.904017905,25
1,nmb,2023-06-12 20:14:38.914020033,8
2,pbg,2023-06-12 20:14:38.914020033,39
3,fie,2023-06-12 20:14:38.914020033,90
4,pfc,2023-06-12 20:14:38.914020033,28


Rows: 8,717,702


# GW Contents

In [11]:
display(gw_head_pdf)
display(gw_tail_pdf)

# Number of Rows
print(f"Rows: {gw_rows:,}")

,sym,time,number
0,boc,2023-04-14 23:17:28.473,53231
1,boc,2023-04-14 23:17:28.473,153560
2,boc,2023-04-14 23:17:28.473,449428
3,boc,2023-04-14 23:17:28.473,631966
4,boc,2023-04-14 23:17:28.473,941566


,sym,time,number
0,ehp,2023-06-12 20:14:39.894016619,32
1,ono,2023-06-12 20:14:39.894016619,49
2,igg,2023-06-12 20:14:39.894016619,56
3,bfj,2023-06-12 20:14:39.894016619,14
4,hcc,2023-06-12 20:14:39.914015502,62


Rows: 34,716,302
